In [1]:
import os
import clip
import json
import spacy
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from PIL import Image
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

In [16]:
# d = []
# with open("./data/description.json", 'r') as f:
#     description = json.load(f)

#     for idx, item in enumerate(description):
#         raw = {}
#         raw['id'] = furniture[idx]['id']
#         raw['description'] = item['description']
#         d.append(raw)

# with open("./data/description.json", 'w') as f:
#     json.dump(d, f, indent=4, ensure_ascii=False)

In [3]:
os.chdir("../")
# os.getcwd()

In [4]:
with open("./data/Furniture.json", 'r', encoding='utf-8-sig') as f:
    furniture = json.load(f)

with open("./data/description.json", 'r') as f:
    description = json.load(f)

print(furniture[20])
print(description[20])

{'id': '2df40e00-b627-4216-90a7-511918861a0f', 'name': 'KIVIK', 'description': '3-seat sofa, light beige', 'pictureUrl': 'https://www.ikea.com/in/en/images/products/kivik-3-seat-sofa-tresund-light-beige__1124111_pe875024_s5.jpg?f=u', 'furnitureModel': 'https://web-api.ikea.com/dimma/assets/geomagical/69484806/PS01_S01_NV_01/simple/glb_draco/0229e9ef62f2d9b7b754e0f00a6fff89-G-69484806-d0cdc207af2b85608ab64ba38df9b679d4916928-simple+draco.glb', 'price': 43990, 'hasOffer': False, 'offerStartDate': None, 'offerEndDate': None, 'discountPercentage': None, 'discountedPrice': None, 'brandId': '7e94f147-004e-4886-8a60-a9f3c98ad1d5', 'categoryId': '3e93f147-004e-4886-8a60-a9f3c98ad1d3', 'styleTypeId': '8e94f147-004e-4886-8a60-a9f3c98ad1d3', 'colorId': '9e94f147-004e-4886-8a60-a9f3c98ad1d9'}
{'description': 'light beige three-seat sofa'}


In [5]:
description[20]["description"]

'light beige three-seat sofa'

In [6]:
import re

nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # Remove punctuation
    text = re.sub(r'[^\w\s]', ' ', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    doc = nlp(text.lower())
    tokens = [
        token.lemma_ for token in doc
    ]
    return " ".join(tokens)

In [7]:
for raw in description:
    raw["description"] = preprocess_text(raw["description"])

description[20]["description"]

'light beige three seat sofa'

In [8]:
import torch
from PIL import Image
import torch.nn.functional as F
from transformers import CLIPProcessor, CLIPModel

# load the feature extraction models
MODEL_PATH = "./models/clip_model/"
model = CLIPModel.from_pretrained(MODEL_PATH)
processor = CLIPProcessor.from_pretrained(MODEL_PATH)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [9]:
text_data = [raw["description"] for raw in description]
text_data

['white dressing table with mirror and drawer',
 'black brown desk with storage drawer',
 'white dressing table with mirror and glass top',
 'white and turquoise storage unit with bin',
 'white storage crate with wheel',
 'white shelf unit with open compartment',
 'toy storage box with wheel',
 'multicolor storage bag with forest animal print',
 'white and red storage unit with drawer',
 'pine wood bed frame',
 'wall mount dollhouse shelf',
 'green and white storage box',
 'white chest with two drawer',
 'white storage unit with box',
 'turquoise storage unit with compartment',
 'white bedside table with shelf',
 'white and pine reversible bed',
 'light wood storage unit with green gray box',
 'grey and turquoise three seat sofa',
 'beige and grey three seat sofa',
 'light beige three seat sofa',
 'light beige three seat sofa with chaise',
 'yellow wing chair',
 'white desk with drawer',
 'dark grey desk',
 'black brown desk',
 'black and blue desk',
 'light grey office chair',
 'dark 

In [10]:
embedding_vector = []

for elm in description:
    text = elm['description']
    # Preprocess text for CLIP
    input = processor(text=text, return_tensors="pt")["input_ids"]

    with torch.no_grad():
        text_features = model.get_text_features(input)

    # convert the features to unit vector
    # cosine similarity only care about the angle between the feature vectors
    text_features = F.normalize(text_features, dim=-1)
    embedding_vector.append(text_features.cpu().numpy())

embedding_vector[0]

array([[-5.62656745e-02, -6.92713400e-03,  6.00755773e-03,
         2.01919582e-03, -1.44139654e-03,  7.94158727e-02,
         1.71957240e-02, -4.68089208e-02, -3.62257957e-02,
        -3.75949107e-02,  2.66860388e-02, -3.58008780e-03,
         2.72734687e-02,  4.32266071e-02, -3.12568527e-03,
        -4.08369564e-02, -3.23890001e-02,  2.57849675e-02,
         7.48147303e-03, -1.72776598e-02,  6.34028092e-02,
        -2.30810568e-02,  3.78929190e-02, -1.63380653e-02,
        -4.34620259e-03, -5.18042333e-02,  5.93704032e-03,
         4.07038536e-03, -4.50178720e-02, -7.42789581e-02,
        -5.66088706e-02,  4.50770408e-02,  3.16585302e-02,
         4.89818491e-03, -1.01515196e-01, -2.24553626e-02,
        -3.96826603e-02, -2.10840208e-03, -5.33078946e-02,
        -6.55469112e-03,  2.34207194e-02,  3.63126844e-02,
         8.88781040e-04,  2.18125675e-02,  5.19574098e-02,
        -9.93722305e-03,  5.18378010e-03,  1.04417568e-02,
         1.42575987e-02, -2.20211409e-02, -5.81601374e-0

In [11]:
embedding_vector = np.array(embedding_vector).astype(np.float32)
embedding_vector.shape

(211, 1, 512)

In [12]:
embedding_vector = embedding_vector.squeeze(axis=1)
embedding_vector.shape

(211, 512)

In [13]:
from dotenv import load_dotenv
import os
load_dotenv()

QDRANT_HOST = os.environ["QDRANT_HOST"]
QDRANT_API_KEY = os.environ["QDRANT_API_KEY"]
client = QdrantClient(
    url=QDRANT_HOST,
    api_key=QDRANT_API_KEY,
    timeout=60.0
)

In [14]:
collection_name = "text_features"

client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=512, distance=Distance.COSINE)
)

/tmp/ipykernel_12804/3742037790.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [15]:
# image_urls = {raw['id']:raw['pictureUrl'] for raw in furniture}
# image_urls

In [16]:
points = []

for idx, (text, feature) in enumerate(zip(text_data, embedding_vector)):
    points.append(
        PointStruct(
            id=idx,
            vector=feature.tolist()
        )
    )

client.upsert(collection_name=collection_name, points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [17]:
query = "swivel chair"

input = processor(text=query, return_tensors="pt")["input_ids"]

with torch.no_grad():
    query_features = model.get_text_features(input)

query_features = F.normalize(query_features, dim=-1)

search_result = client.search(
    collection_name=collection_name,
    query_vector=query_features[0].tolist(),
    limit=10
)

/tmp/ipykernel_12804/1393974746.py:10: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(


In [20]:
for hit in search_result:
    print("Score:", hit.score)
    print("ID:", hit.id)
    print("Description:", furniture[hit.id]["pictureUrl"])
    print("-"*20)

Score: 0.92210555
ID: 38
Description: https://www.ikea.com/in/en/images/products/skruvsta-swivel-chair-ysane-white__0724712_pe734595_s5.jpg?f=u
--------------------
Score: 0.89243877
ID: 37
Description: https://www.ikea.com/in/en/images/products/skruvsta-swivel-chair-vissle-grey__0724705_pe734587_s5.jpg?f=u
--------------------
Score: 0.8576086
ID: 56
Description: https://www.ikea.com/in/en/images/products/teodores-chair-black__1114240_pe871696_s5.jpg?f=u
--------------------
Score: 0.8576086
ID: 73
Description: https://www.ikea.com/in/en/images/products/adde-chair-black__0728277_pe736167_s5.jpg?f=u
--------------------
Score: 0.85078776
ID: 35
Description: https://www.ikea.com/in/en/images/products/flintan-office-chair-with-armrests-black__1007241_pe825960_s5.jpg?f=u
--------------------
Score: 0.8424226
ID: 36
Description: https://www.ikea.com/in/en/images/products/flintan-office-chair-with-armrests-beige__1007238_pe825958_s5.jpg?f=u
--------------------
Score: 0.83650994
ID: 71
Desc